In [87]:
import numpy as np
import pandas as pd
import pyro
import itertools

print_it = False

weather = pd.read_csv("/Users/asherkite/Desktop/School/Courses/MBML/MBML_Project/MBML_github/MBML-traffic-accidents/data/2023_Weather.csv")
accident = pd.read_csv("/Users/asherkite/Desktop/School/Courses/MBML/MBML_Project/MBML_github/MBML-traffic-accidents/Asher/accident_data.csv")
#cleaning up the data
accident["datetime"] = pd.to_datetime(accident["datetime"])
accident.set_index(['cell_id', 'datetime'], inplace=True)
accident = accident.sort_index()
if print_it: print("accident index:", accident.index.names)
weather.rename(columns={"Time":"datetime"}, inplace=True)
weather['datetime'] = pd.to_datetime(weather['datetime'])
weather = weather.drop("Unnamed: 0",axis=1)
weather.set_index("datetime",inplace= True)
if print_it: print("weather index:", weather.index.names)
#preparing the multiindex
cell_ids = accident.index.get_level_values('cell_id').unique()
datetimes = weather.index.unique() 
full_index = pd.MultiIndex.from_product(
    [cell_ids, datetimes],
    names=['cell_id', 'datetime']
)
#prepare the merged dataframe
df = pd.DataFrame(index=full_index)
df = df.sort_index()
if print_it: print("df index:", df.index.names)
df = df.join(accident)
df = df.reset_index()
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.merge(
    weather.reset_index(), on='datetime', how='left'
).set_index(['cell_id', 'datetime'])
df['acc_count'] = df['acc_count'].fillna(0)
if print_it: print("accident rows == 1:", (accident['acc_count'] == 1).sum())
if print_it: print("df rows == 1:",(df['acc_count'] == 1).sum())

merged = df


In [84]:
df.to_csv ("/Users/asherkite/Desktop/School/Courses/MBML/MBML_Project/MBML_github/MBML-traffic-accidents/Asher/merged.csv")
print(df)

                                                X   Y  acc_count  acc_precip  \
cell_id                   datetime                                             
(717514.445, 6177963.565) 2023-01-01 00:00:00 NaN NaN        0.0         1.1   
                          2023-01-01 01:00:00 NaN NaN        0.0         3.5   
                          2023-01-01 02:00:00 NaN NaN        0.0         0.9   
                          2023-01-01 03:00:00 NaN NaN        0.0         2.3   
                          2023-01-01 04:00:00 NaN NaN        0.0         0.7   
...                                            ..  ..        ...         ...   
(729014.445, 6173963.565) 2023-12-31 19:00:00 NaN NaN        0.0         0.3   
                          2023-12-31 20:00:00 NaN NaN        0.0         1.2   
                          2023-12-31 21:00:00 NaN NaN        0.0         1.6   
                          2023-12-31 22:00:00 NaN NaN        0.0         2.7   
                          2023-12-31 23:

Everything below this I am not confident about:

In [110]:
import pandas as pd
import torch
import ast


# Load your already-prepared DataFrame
df = merged.copy()  # Replace with however you loaded it

# Reset index to access 'cell_id'
df = df.reset_index()

df['datetime'] = pd.to_datetime(df['datetime'])

# Filter to December 2023
df = df[df['datetime'].dt.month == 12]


# Parse EPSG:25832 coordinates from cell_id strings
df[["X", "Y"]] = df["cell_id"].apply(lambda s: pd.Series(ast.literal_eval(s)))

# Drop rows with missing values
df = df.dropna(subset=["X", "Y", "acc_precip", "mean_temp", "mean_relative_hum", "mean_radiation"])

# Normalize weather features (important for GP + stability)
from sklearn.preprocessing import StandardScaler
weather_cols = ["acc_precip", "mean_temp", "mean_relative_hum", "mean_radiation"]
scaler = StandardScaler()
df[weather_cols] = scaler.fit_transform(df[weather_cols])

# Convert to torch tensors
X_space = torch.tensor(df[["X", "Y"]].values, dtype=torch.float32)
X_weather = torch.tensor(df[weather_cols].values, dtype=torch.float32)
y = torch.tensor(df["acc_count"].values, dtype=torch.float32)


In [112]:
import pyro
import pyro.distributions as dist
import pyro.contrib.gp as gp
from pyro.infer import SVI, Trace_ELBO
import pyro.optim

# Kernel for GP over 2D spatial input
kernel = gp.kernels.RBF(input_dim=2)
Xu = X_space[:500]  # Optional: use a subset for inducing points (speed)

# Sparse GP for scalability (use VariationalSparseGP instead of full GP)
gp_model = gp.models.VariationalSparseGP(X_space, None, kernel, Xu, likelihood=None)

def model(X_weather, y):
    beta = pyro.sample("beta", dist.Normal(0., 1.).expand([X_weather.shape[1]]).to_event(1))
    f_x, _ = gp_model.model()  # Get mean and varianc  # latent GP samples (num_data,)
    log_lambda = f_x + X_weather @ beta
    lambda_ = torch.exp(log_lambda)

    with pyro.plate("data", len(y)):
        pyro.sample("obs", dist.Poisson(lambda_), obs=y)

def guide(X_weather, y):
    gp_model.guide()
    beta_loc = pyro.param("beta_loc", torch.zeros(X_weather.shape[1]))
    beta_scale = pyro.param("beta_scale", torch.ones(X_weather.shape[1]), constraint=dist.constraints.positive)
    pyro.sample("beta", dist.Normal(beta_loc, beta_scale).to_event(1))


In [115]:
pyro.clear_param_store()
optimizer = pyro.optim.Adam({"lr": 0.01})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# Optional: Subsample for early tests
n_steps = 2000
for step in range(n_steps):
    loss = svi.step(X_weather, y)
    if step % 10 == 0:
        print(f"[{step}] ELBO: {loss:.2f}")


_LinAlgError: linalg.cholesky: The factorization could not be completed because the input is not positive-definite (the leading minor of order 481 is not positive-definite).
     Trace Shapes:        
      Param Sites:        
                Xu 500 2  
kernel.lengthscale        
   kernel.variance        
     Sample Sites:        
         beta dist     | 4
             value     | 4